# Krylov Subspace Approximation XL-BOMD (KSA-XL-BOMD)
### see https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import KSA_XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/5_KSA_XL-BOMD'}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params,
                   seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)

md.initialize_velocity(molecule,  Temperature=400.0)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   504.34   2.607617e-01 -1.360431e+00 -1.099670e+00 ||    381.70   1.973523e-01 1.419623e+00 1.616976e+00 || 
     2   475.53   2.458703e-01 -1.346387e+00 -1.100517e+00 ||    425.24   2.198677e-01 1.396692e+00 1.616559e+00 || 
     3   427.62   2.210955e-01 -1.321380e+00 -1.100285e+00 ||    480.49   2.484311e-01 1.367701e+00 1.616132e+00 || 
     4   367.50   1.900123e-01 -1.289177e+00 -1.099165e+00 ||    537.55   2.779351e-01 1.337880e+00 1.615815e+00 || 
     5   301.89   1.560900e-01 -1.254880e+00 -1.098790e+00 ||    585.02   3.024771e-01 1.313005e+00 1.615482e+00 || 
     6   238.24   1.231794e-01 -1.221743e+00 -1.098564e+00 ||    612.29   3.165760e-01 1.298633e+00 1.615209e+00 || 
     7   184.00   9.513614e-02 -1.193923e+00 -1.098787e+00 ||    612.30   3.165837e-01 1.298415e+00 1.614999e+00 || 
     8   144.30   7.460697e-02 -1.173484e+00 -1.098877e+00 ||    583.66   3.017773e-01 1.313245e+00 1.615

## Damping (Langevin)

In [3]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/5_KSA_XL-BOMD_damp'}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)

md.initialize_velocity(molecule, Temperature=400.0)
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   503.18   2.601635e-01 -1.360614e+00 -1.100450e+00 ||    385.34   1.992351e-01 1.420770e+00 1.620005e+00 || 
     2   487.24   2.519228e-01 -1.346048e+00 -1.094125e+00 ||    428.79   2.217040e-01 1.401173e+00 1.622877e+00 || 
     3   439.08   2.270237e-01 -1.320263e+00 -1.093239e+00 ||    474.28   2.452212e-01 1.373082e+00 1.618303e+00 || 
     4   367.25   1.898833e-01 -1.284998e+00 -1.095115e+00 ||    512.55   2.650065e-01 1.345550e+00 1.610557e+00 || 
     5   290.17   1.500308e-01 -1.249427e+00 -1.099396e+00 ||    536.81   2.775546e-01 1.320119e+00 1.597673e+00 || 
     6   218.90   1.131801e-01 -1.214222e+00 -1.101042e+00 ||    540.33   2.793742e-01 1.301954e+00 1.581329e+00 || 
     7   161.78   8.364468e-02 -1.185536e+00 -1.101891e+00 ||    539.90   2.791498e-01 1.294345e+00 1.573495e+00 || 
     8   139.60   7.217641e-02 -1.167021e+00 -1.094844e+00 ||    533.10   2.756364e-01 1.299874e+00 1.575